# Collect and Process Twitter Feeds for Data Analysis: the Pythonic Way

## OSINT with Python and ELKstack [Part 1]
> Andrew Eng | 2020-10-05 | Part 1 

Updated: 2020-10-09:
- Cleaned up code blocks to make sure it runs without errors

**This article was renamed from Open Source Intelligence with Elasticsearch: Analyzing Twitter Feeds 1/3 as the title no longer fit the description of the series.**

## Introduction
This is a three part series in open source intelligence with Elasticsearch.

In part 1: We will look at data acquisition and building a python script to acquire, clean, and transfer data.

In part 2: We will look at building visualizations and dashboards in Kibana and Elasticsearch.

In Part 3: We will look at operationalizing the script into sustainment.

## Start with Why
I had an opportunity to interview with a large tech company but I didn’t know much about them (I’ve been oblivious to what’s popping up in Silicon Valley). Following due diligence and better prepare for the interview, I spent the weekend working on script to help me better understand the general sentiments and what the company was about. At that point in time, any news is new news, and I wanted to consume it.

Twitter is a great social networking platform that delivers almost real-time news generated and posted by “regular people”. It’s great, because it’s really unfiltered and you’re able to somewhat grasp what people are thinking. **The goal of this project is to build a twitter scrapper, dump it into Elasticsearch and create visualization dashboards using Kibana**.

Like most people, I’m a visual person. I love looking at dashboards, graphs, and pictures that makes sense of data. It helps me comprehend and gain insight in a particular subject. I’m also a project based learner. I learn better when I have an end goal. In this case, I’m learning: Python, Elasticsearch, and Kibana. I’m also learning how to use the Twitter API for other projects that I’m working on like Natural Language Processing (Sentiment Analysis, bots, docker, etc.)

### Visualizations: What questions are we trying to answer?
My main focus is to collect as much information about my search as possible. I’d like to track trending tweets, stories, and what people are generally thinking about the subject. With this, I can probably track “influencers. I’m interested in the social impact on how influencers influence. My initial questions to answer are:

1. What are the most retweets of the specific search term? (Tracking topic popularity)
2. How many unique users are tweeting about a given search term? (Is it trending?)
3. What other tweets are they posting about the search term? (Are they a troll / bot?)
4. How many likes does the person have on the search term? (Trending / Sentiments)?

## My Environment
It really doesn’t matter what I’m using, because the processing is low overhead. It’s not processing intensive calculations or stitching anything together. It’s simple: grab tweets, format it into JSON or dictionary, and send it off to Elasticsearch. For this environment, I’ll be using docker to quickly stand-up an Elasticsearch and Kibana (ELKSTACK) so I can focus on the coding parts.

### Docker

<code>$ git clone https://github.com/andreweng/elkstack.git</code>

<code>$ cd elkstack</code>

<code>$ cd docker-compose up</code>

![docker-compose up](images/docker-compose_up.png)

Afterwards, test you have ELKSTACK up and running by browsing to: **[http://localhost:5601](http://localhost:5601)**

![initial_kibana](images/initial_kibana.png)

### Python
For my twitter scrapper, I’ll be using python 3.8.3 and my IDE will be Jupyter Notebook. Jupyter Notebook is a great Integrated Development Environment that allows you to write code in blocks and run it individually within a web browser. It’s easy to initiate and secure enough to get the job done and tear it down. By default, Jupyter will create open port 8888 on localhost (127.0.0.1) with unique tokens every time you initiate it.

<code>$ cd ~/elkstack/scripts</code>

<code>$ jupyter-notebook</code>

![jupyter_initiate](images/jupyter_initiate.png)

The command above is run on a terminal window while inside the elkstack/scripts folder.

## Coding
### Twitter API
Since we are going to programmatically access twitter and pull records, we’ll need to use the Twitter API. You can grab your APIs following the Twitter API link [https://developer.twitter.com/en](https://developer.twitter.com/en)

#### Authentication
There are 3 types of authentication mechanisms we can use, however basic authentication is deprecated for awhile now:

- **OAuth 1.0a is a method used to make API requests on behalf of a Twitter Account. In other words, application-user authentication.**
- OAuth 2.0 Bearer Token is a authenticates requests on behalf of your developer App. Specifically, it allows your app to pull records in READ-ONLY. In other words, application authentication.
- Basic Authentication accesses twitter as you, it’s basic authentication where you pass your email and password to the app [no longer in use].

For this use-case, I’ll be using the **OAuth 1.0a**, this is mainly so I can access a full range of APIs for later scalability.

![Twitter_Auth_API](images/twitter-auth-api.png)

### Python Modules

**Quick install of modules:**

<code>$ sudo pip3 install -r requirements.txt</code>

Let's take a look at what modules we are using.  If you used requirements.txt to install the modules, you don't need to run the below commands.

**External modules:**

[Tweepy](https://github.com/tweepy/tweepy) **|** [Documentation](http://docs.tweepy.org/en/latest/index.html)  

I created a notebook that I used to take notes and play around with the module [Tweepy_tutorial](tweepy_tutorial.ipy)

[Elasticsearch](https://github.com/elastic/elasticsearch-py) **|** [Documentation](https://elasticsearch-py.readthedocs.io/en/master/)

**Built in modules:**

[json Documentation](https://docs.python.org/3/library/json.html)

[datetime Documentation](https://docs.python.org/3/library/datetime.html)

<code>$ sudo pip3 install tweepy elasticsearch</code>

![pip3_install_tweepy](images/tweepy_install.png)
![pip3_install_elasticsearch](images/elasticsearch_install.png)

### Execute
Import the modules we be using

In [8]:
import tweepy as tw
from datetime import datetime as dt
from elasticsearch import Elasticsearch

Generally, it's not a good idea to include credentials and access tokens in scripts.  There's additional security stuff we can explore when we operationalize the script, but for now, let's get this up and running.  Create your API credentials text file and name it twitter.keys. My twitter.keys file looks like:

```text
(base) andrew@tangent:~$ cat twitter.keys 
api_key = <redacted>
api_secret_key = <redacted>
access_token = <redacted>
access_token_secret = <redacted>
(base) andrew@tangent:~$ 
```

Import keys into the script

In [10]:
# Import keys from a saved file instead of inputting it directly into the script
key_location = 'twitter.keys'
apikeys = []
with open(key_location) as keys:
    for items in keys:
        apikeys.append(items.split('=')[1].strip(' ').strip('\n'))
    keys.close()

Pass the key information to the script and set server configurations

In [11]:
# Initialize dictionary
twitter_cred = dict()

# Enter API keys
twitter_cred["CONSUMER_KEY"] = apikeys[0]
twitter_cred["CONSUMER_SECRET"] = apikeys[1]

# Access Tokens
twitter_cred["ACCESS_KEY"] = apikeys[2]
twitter_cred["ACCESS_SECRET"] = apikeys[3]

auth = tw.OAuthHandler(twitter_cred["CONSUMER_KEY"], twitter_cred["CONSUMER_SECRET"])

auth.set_access_token(twitter_cred["ACCESS_KEY"], twitter_cred["ACCESS_SECRET"])

api = tw.API(auth, wait_on_rate_limit=True)

# Initialize elasticsearch node
es = Elasticsearch('127.0.0.1', port=9200)

**Hello World!** Let’s test our API to see if our authentication is working

In [12]:
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

New York Times @nytimes: Facebook and the Group That Planned to Kidnap Gretchen Whitmer. #dataresponsible #AI… https://t.co/rM9pYLyxJP
The $25M class action against Tezos was a 'lawyers' picnic' ... that has now descended into an unseemly brawl over… https://t.co/7Klirw1Tvf
Artificial Intelligence research at University of Adelaide gets $20 million boost. #dataresponsible #robotics… https://t.co/IM6ZwyHQUW
Abuse applications, impersonate users, escalate privileges and fiddle with #ActiveDirectory permissions in our chal… https://t.co/LuZ1KfthlK
The recent @TheJusticeDept framework on cryptocurrency enforcement has taken flak from many in the crypto community https://t.co/sqYm7cJ7mQ
RT @LocalCryptosEN: Every #Dash trade so far has used #InstantSend, making trading lightning-fast. ⚡️⚡️⚡️

Because LocalCryptos accepts zer…
Learn how to perform process injection into a process with various injection techniques. #PentesterAcademy… https://t.co/a7u8c24Nla
This week, #Bitcoin stays resilient,

Data acquisition function: Takes 2 arguments; 
- search = what to search for, 
- acq = how many tweets to grab

> usage: acqData('palantir','100')

There’s a lot of metadata in each tweet. I am only extracting ones that I care about. In actuality, it might be better to collect everything and perform post processing after it hits Elasticsearch.

In [19]:
def acqData(search, acq):

    # Create an index name using our search criteria and today's date
    index_name = search.split(' ')[0] + '-' + dt.today().strftime('%Y-%m-%d')

    # Initialize the feed list of dictionaries
    feed = []
    
    print('::Acquiring Data::')
   
    # Data Acquisition
    for tweet in tw.Cursor(api.search, q=search, tweet_mode='extended').items(acq):
        feed.append(tweet._json)

    # Formatting the data and extracting what we need
    count = 0
    
    print('::Transferring to Elasticsearch Search::')
    
    while count < len(feed):
        # Created variables instead of directly injecting it to the dictionary because it's easier to read
        tweet_date = feed[count]['created_at']
        username = feed[count]['user']['screen_name']
        account_creation_date = feed[count]['created_at']
        user_description = feed[count]['user']['description']
        user_url = feed[count]['user']['url']
        verified_status = feed[count]['user']['verified']
        geo_enabled = feed[count]['user']['geo_enabled']
        friends_count = feed[count]['user']['friends_count']
        followers_count = feed[count]['user']['followers_count']
        retweeted_count = feed[count]['retweet_count']
        favorite_count = feed[count]['favorite_count']
        hashtags = feed[count]['entities']['hashtags']
        tweet_full_text = feed[count]['full_text']

        # Prepare data for elasticsearch
        doc = {
            '@timestamp': dt.now(),
            'tweet_date': tweet_date,
            'username': str(username),
            'account_creation_date': str(account_creation_date),
            'user_description': str(user_description),
            'user_url': str(user_url),
            'verified_status': bool(verified_status),
            'geo_enabled': bool(geo_enabled),
            'friends_count': int(friends_count),
            'followers_count': int(followers_count),
            'retweeted_count': int(retweeted_count),
            'favorite_count': int(favorite_count),
            'hashtags': hashtags,
            'tweet_full_text': str(tweet_full_text),
            'word_list': str(tweet_full_text).split(' ')
        }

        # Import into elasticsearch using the generated index name at the top of the function <search> + <date>
        es.index(index=index_name, body=doc)

        count +=1

In the below code, we are calling the acqData function and passing the arguments ‘palantir OR PLTR’ and 100. I am looking for any keyword matches for “palantir OR PLTR” and I want to grab 100 most recent tweets.

In [20]:
# Main Function; Let's run this function!
acqData('palantir OR PLTR', 100)

::Acquiring Data::
::Transferring to Elasticsearch Search::


![AcqData Sample Image](images/acqData_sample1.png)

We now have the data in the format we want and it imported into Elasticsearch using the es.index method in our acqData function. Let’s check Kibana.

![Screenshot of data imported into Elasticsearch](images/es_stage1.png)

## Parting Thoughts:
Part 1 of this series focuses on data acquisition and using python along with 2 modules: Tweepy and Elasticsearch. We used Tweepy as programmatic interface to Twitter and then used Elasticsearch to inject tweets into the database. Looking at Kibana, it already has some really useful information:

- Percentage of usernames that tweeted about the topic in the 2500 tweet download
- Percentage of retweets found in the 2500 tweet download

In Part 2, we’ll explore the data a little bit more and try to answer some of the questions that we initially came up with. We’ll create dashboards and visualizations and explore what else we can extract from the information we got.